In [ ]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 13.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 58.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 65.8 MB/s eta 0:00:0

In [ ]:
!nvidia-smi

/bin/bash: nvidia-smi: command not found


In [ ]:
###allocating device

import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
##Import Necessary Libraries

from transformers import pipeline
import matplotlib.pyplot as plt
from datasets import load_dataset, load_metric
import pandas as pd

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk
import torch

nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
####Loading Pubmed Dataset for Scientifc Articles

dataset_pubmed = load_dataset("scientific_papers","pubmed")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/119924 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6633 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6658 [00:00<?, ? examples/s]

Dataset scientific_papers downloaded and prepared to /root/.cache/huggingface/datasets/scientific_papers/pubmed/1.1.1/306757013fb6f37089b6a75469e6638a553bd9f009484938d8f75a4c5e84206f. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
###This block of code is not needed to execute.Just a demo
##text = "anemia occurs in more than 50% of patients with cancer and may reach 90% in certain types of cancers , especially in patients undergoing chemotherapy and/or radiation therapy . <n> "
##n = text.replace(" . <n> ", "\n")
##print(n)

In [ ]:
###Loading features of Pubmed dataset stored as dictionary

dataset_pubmed.keys()
dataset_pubmed.values()

dict_values([Dataset({
    features: ['article', 'abstract', 'section_names'],
    num_rows: 119924
}), Dataset({
    features: ['article', 'abstract', 'section_names'],
    num_rows: 6633
}), Dataset({
    features: ['article', 'abstract', 'section_names'],
    num_rows: 6658
})])

In [ ]:
list(dataset_pubmed.items())

[('train',
  Dataset({
      features: ['article', 'abstract', 'section_names'],
      num_rows: 119924
  })),
 ('validation',
  Dataset({
      features: ['article', 'abstract', 'section_names'],
      num_rows: 6633
  })),
 ('test',
  Dataset({
      features: ['article', 'abstract', 'section_names'],
      num_rows: 6658
  }))]

In [ ]:
dataset_pubmed["train"][1]["section_names"]

'Introduction\nPatients and methods\nStudy design\nTreatment\nAssessment\nStatistical analysis\nResults\nDiscussion\nConclusion'

In [ ]:
###Taking 1st piece of Article from Train Dataset

sample_dataset = dataset_pubmed["train"][1]["article"]
sample_dataset

'it occurs in more than 50% of patients and may reach 90% in certain types of cancers , especially in patients undergoing chemotherapy and/or radiation therapy.1 anemia is defined as an inadequate circulating level of hemoglobin ( hb ) ( hb < 12 g / dl ) and may arise as a result of the underlying disease , bleeding , poor nutrition , chemotherapy , or radiation therapy .\npreliminary studies suggest that survival and loco - regional control after radiation therapy , especially in head and neck cancers , may be compromised by anemia.24 anemia often worsens symptoms such as fatigue , weakness , and dyspnea , and thus may have a negative effect on quality of life ( qol ) and performance status in patients with cancer .\nthus , to improve physical functioning , qol , and prognosis in patients with cancer , it would be reasonable to take a proactive approach in identifying populations who need treatment for cancer - associated anemia ( caa ) and provide timely management .\nblood transfusi

In [ ]:
words_sampledataset = sample_dataset.split()
print('Number of words in the article is:', len(words_sampledataset))

Number of words in the article is: 3258


# Using Hugging Face's Pegasus - Pubmed Summarization Pipeline to generate the Summary from the Model

In [ ]:
###Import PegasusModel and Tokenizer

from transformers import pipeline, PegasusTokenizer, PegasusForConditionalGeneration


# Named entity recognition pipeline, passing in a specific model and tokenizer
model = PegasusForConditionalGeneration.from_pretrained('google/pegasus-pubmed')
tokenizer =PegasusTokenizer.from_pretrained('google/pegasus-pubmed')

summerize_pipe = pipeline("summarization", model=model, tokenizer=tokenizer)
pipe_out = summerize_pipe(sample_dataset, truncation=True)
pipe_out

[{'summary_text': 'anemia occurs in more than 50% of patients with cancer and may reach 90% in certain types of cancers , especially in patients undergoing chemotherapy and/or radiation therapy . <n> this pilot study assessed the efficacy and feasibility of intravenous ( iv ) iron monotherapy in patients with cancer who have anemia and who are undergoing treatment with chemotherapy and/or radiation therapy without the use of erythropoiesis - stimulating agents ( esas ) . <n> eligible patients were at least 18 years old , about to start a cycle of chemotherapy and/or radiation therapy within 1 week of inclusion , and had a nonmyeloid malignancy , hemoglobin levels of 11.0 g / dl or less , a life expectancy of more than 24 weeks , and an eastern cooperative oncology group performance status of 02 . <n> patients were also required to have a serum ferritin level of 100 ng / ml or higher or transferrin saturation ( tsat ) levels of 15% or higher and to have received no esas or iv iron thera

In [ ]:
###Dictionary to store Results of summary of different Models

summaries = {}
summaries["pegasus"] = pipe_out[0]["summary_text"].replace(" . <n> ", "\n")
print(summaries["pegasus"])

anemia occurs in more than 50% of patients with cancer and may reach 90% in certain types of cancers , especially in patients undergoing chemotherapy and/or radiation therapy
this pilot study assessed the efficacy and feasibility of intravenous ( iv ) iron monotherapy in patients with cancer who have anemia and who are undergoing treatment with chemotherapy and/or radiation therapy without the use of erythropoiesis - stimulating agents ( esas )
eligible patients were at least 18 years old , about to start a cycle of chemotherapy and/or radiation therapy within 1 week of inclusion , and had a nonmyeloid malignancy , hemoglobin levels of 11.0 g / dl or less , a life expectancy of more than 24 weeks , and an eastern cooperative oncology group performance status of 02
patients were also required to have a serum ferritin level of 100 ng / ml or higher or transferrin saturation ( tsat ) levels of 15% or higher and to have received no esas or iv iron therapy within 30 days and no oral iron th

# *Evaluating Generated Summary from Training Split Pegasus-Pubmed Model using ROUGE Score.*

In [ ]:
rouge_metric = load_metric('rouge')

<ipython-input-15-90542a62301a>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric('rouge')


In [ ]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

reference = dataset_pubmed['train'][1]['abstract']

records = []

for model_name in summaries:
    rouge_metric.add(prediction = summaries[model_name], reference = reference )
    score = rouge_metric.compute()
    rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )
    print('rouge_dict ', rouge_dict )
    records.append(rouge_dict)

pd.DataFrame.from_records(records, index = summaries.keys() )

rouge_dict  {'rouge1': 0.3600782778864971, 'rouge2': 0.14538310412573674, 'rougeL': 0.23091976516634052, 'rougeLsum': 0.34442270058708413}


,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.360078,0.145383,0.23092,0.344423


# *Using Text Summarization Pipeline To generate Summary Results from Test Dataset*

In [ ]:
from transformers import pipeline, PegasusTokenizer, PegasusForConditionalGeneration


# Named entity recognition pipeline, passing in a specific model and tokenizer
model = PegasusForConditionalGeneration.from_pretrained('google/pegasus-pubmed')
tokenizer =PegasusTokenizer.from_pretrained('google/pegasus-pubmed')

summerize_pipe = pipeline("summarization", model=model, tokenizer=tokenizer)
pipe_out = summerize_pipe(dataset_pubmed['test'][0]['article'], truncation=True)
pipe_out

[{'summary_text': "anxiety is the most prominent and prevalent mood disorder in parkinson 's disease ( pd ) ; however , little is known about the relationship between anxiety and cognition in pd . <n> the aim of this study was to examine the influence of anxiety on cognition in pd by directly comparing groups of pd patients with and without anxiety while excluding depression . <n> we hypothesized that pd patients with anxiety would show impairments in attentional set - shifting and working memory compared to pd patients without anxiety . <n> furthermore , since previous work , albeit limited , has focused on the influence of symptom laterality on anxiety and cognition , we also explored this relationship . <n> seventeen pd patients with anxiety and thirty - three pd patients without anxiety were included in this study . <n> results showed that pd patients with anxiety had worse attentional set - shifting and working memory performance than pd patients without anxiety . <n> in addition 

In [ ]:
summaries = {}
summaries["pegasus"] = pipe_out[0]["summary_text"].replace(" .<n>", ".\n")
print(summaries["pegasus"])

anxiety is the most prominent and prevalent mood disorder in parkinson 's disease ( pd ) ; however , little is known about the relationship between anxiety and cognition in pd . <n> the aim of this study was to examine the influence of anxiety on cognition in pd by directly comparing groups of pd patients with and without anxiety while excluding depression . <n> we hypothesized that pd patients with anxiety would show impairments in attentional set - shifting and working memory compared to pd patients without anxiety . <n> furthermore , since previous work , albeit limited , has focused on the influence of symptom laterality on anxiety and cognition , we also explored this relationship . <n> seventeen pd patients with anxiety and thirty - three pd patients without anxiety were included in this study . <n> results showed that pd patients with anxiety had worse attentional set - shifting and working memory performance than pd patients without anxiety . <n> in addition , pd patients with 

# **Evaluating Summaries Generaeted From Test Dataset**

In [ ]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

reference = dataset_pubmed['test'][0]['abstract']

records = []

for model_name in summaries:
    rouge_metric.add(prediction = summaries[model_name], reference = reference )
    score = rouge_metric.compute()
    rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )
    print('rouge_dict ', rouge_dict )
    records.append(rouge_dict)

pd.DataFrame.from_records(records, index = summaries.keys() )

rouge_dict  {'rouge1': 0.46109510086455324, 'rouge2': 0.16231884057971013, 'rougeL': 0.23631123919308358, 'rougeLsum': 0.39193083573487036}


,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.461095,0.162319,0.236311,0.391931


# *T5 Summarization Pipeline to generate the Summary of Pubmed Articles*

In [ ]:
pipe_T5 = pipeline('summarization', model = 't5-small' )

pipe_out_T5 = pipe_T5(dataset_pubmed['test'][0]['article'])


Token indices sequence length is longer than the specified maximum sequence length for this model (4619 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
pipe_out_T5

[{'summary_text': "pd patients with parkinson 's disease were more affected by anxiety than lpd . apathy and anhedonia have been associated with cognitive impairments . the study was carried out by a neurologist at the university of sydney . it has been suggested that anxiety may disrupt compensatory striatal processes ."}]

In [ ]:
from nltk.tokenize import sent_tokenize
summaries['t5'] = 'n'.join(sent_tokenize(pipe_out_T5[0]['summary_text']))

# *BART Implementation*

In [ ]:
from transformers import pipeline, BartTokenizer, BartForConditionalGeneration


# Named entity recognition pipeline, passing in a specific model and tokenizer
model_bart = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
tokenizer_bart =BartTokenizer.from_pretrained('facebook/bart-base')

summerize_pipe_bart = pipeline("summarization", model='facebook/bart-base' , tokenizer=tokenizer_bart)
pipe_out_bart = summerize_pipe_bart(dataset_pubmed['test'][0]['article'], truncation=True)
pipe_out_bart

[{'summary_text': "anxiety affects quality of life in those living with parkinson 's disease ( pd ) more so than overall cognitive status , motor deficits , apathy , and depression [ 13 ] . [ 14 ]although anxiety and depression are often related and coexist in pd patients , recent research suggests that anxiety rather than depression is the most prominent and prevalent mood disorder in p'd [ 5 , 6 ] . yet , the current understanding of anxiety and its impact on cognition in the pd , as well as its neural basis and best treatment practices , remains meager and lags far behind that of depression . [ 15 ] . ."}]

In [ ]:
summaries["bart"] = "\n".join(sent_tokenize(pipe_out_bart[0]["summary_text"]))

In [ ]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

reference = dataset_pubmed['test'][0]['abstract']

records = []

for model_name in summaries:
    rouge_metric.add(prediction = summaries[model_name], reference = reference )
    score = rouge_metric.compute()
    rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )
    print('rouge_dict ', rouge_dict )
    records.append(rouge_dict)

pd.DataFrame.from_records(records, index = summaries.keys() )

rouge_dict  {'rouge1': 0.46109510086455324, 'rouge2': 0.16231884057971013, 'rougeL': 0.23631123919308358, 'rougeLsum': 0.39193083573487036}
rouge_dict  {'rouge1': 0.19555555555555557, 'rouge2': 0.05381165919282511, 'rougeL': 0.11555555555555555, 'rougeLsum': 0.15111111111111108}
rouge_dict  {'rouge1': 0.2867647058823529, 'rouge2': 0.07407407407407407, 'rougeL': 0.15441176470588233, 'rougeLsum': 0.23529411764705882}


,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.461095,0.162319,0.236311,0.391931
t5,0.195556,0.053812,0.115556,0.151111
bart,0.286765,0.074074,0.154412,0.235294


# *Explanations*

Keyword extraction : https://github.com/summanlp/textrank

Comment: very old paper; not very intuitive

In [ ]:
!pip install summa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for summa: filename=summa-1.2.0-py3-none-any.whl size=54389 sha256=945a3f38d2a875d0f09740c124193fcb7a640cdc8d92fd04c722f8cb5330bf35
  Stored in directory: /root/.cache/pip/wheels/4a/ca/c5/4958614cfba88ed6ceb7cb5a849f9f89f9ac49971616bc919f
Successfully built summa


In [ ]:
from summa import summarizer

In [ ]:
data_for_test = dataset_pubmed['test'][0]['article']

In [ ]:
print(summarizer.summarize(data_for_test))

anxiety affects quality of life in those living with parkinson 's disease ( pd ) more so than overall cognitive status , motor deficits , apathy , and depression [ 13 ] .
although anxiety and depression are often related and coexist in pd patients , recent research suggests that anxiety rather than depression is the most prominent and prevalent mood disorder in pd [ 5 , 6 ] .
however , few studies have specifically investigated the relationship between anxiety and cognition in pd .
one study showed a strong negative relationship between anxiety ( both state and trait ) and overall cognitive performance ( measured by the total of the repeatable battery for the assessment of neuropsychological status index ) within a sample of 27 pd patients .
two further studies have examined whether anxiety differentially affects cognition in patients with left - sided dominant pd ( lpd ) versus right - sided dominant pd ( rpd ) ; however , their findings were inconsistent .
the first study found that 

In [ ]:
from summa import keywords
print(keywords.keywords(data_for_test))

pd
anxiety affects
cognition
tests
test
memory
u
patients
patient
p
scores
scored
depression
depressed
neural
studies
study
cognitive status
working
attention
attentional
researchers
group
significantly
significant
comparing groups
performance
performed
perform
compared
compare
remains
remained
functioning
functional
function
verbally
verbal
score greater
related
relatively
relating
reported
processing
sample
findings
recent research
participants
compensatory processes
disease
previous work
iii
disorder
disorders
dopamine
executive functions
items
chronic
affective
affected
affect
future
hads
recently
assessment
assess
assessing
treatment
domains assessed
scale
scales
domain
reuptake
criteria
lpd
neuropsychological
symptoms
symptom
impairs
impairments
impaired
impair
impairment
measured
measures
measure
effect
effects
tasks
task
span
battery
coeruleus
psychotropic
sectional
centre
informed
hippocampal
previously
z
mmse
negatively
negative
key
deficits
higher
influence
influences
mediat

# *SHAP XAI for Text Summarization: Text to Text Explanation: Abstractive Summarization Example*

In [ ]:
!pip install shap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.6/572.6 kB 9.3 MB/s eta 0:00:00


In [ ]:
import numpy as np
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
from datasets import load_dataset
import shap

In [ ]:
explainer = shap.Explainer(model, tokenizer) #creating an explainer object

In [ ]:
dataset = load_dataset('xsum', split='train')
s = dataset['document'][0:1]
print(s)

Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

Dataset xsum downloaded and prepared to /root/.cache/huggingface/datasets/xsum/default/1.2.0/082863bf4754ee058a5b6f6525d0cb2b18eadb62c7b370b095d1364050a52b71. Subsequent calls will reuse this data.
['The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.\nRepair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.\nTrains on the west coast mainline face disruption due to damage at the Lamington Viaduct.\nMany businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.\nFirst Minister Nicola Sturgeon visited the area to inspect the damage.\nThe waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.\nJeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.\nHowever, she said more preventative 

In [ ]:
whole_article=np.array([data_for_test])
real_abstract= dataset_pubmed['test'][0]['abstract']
abstract=np.array([real_abstract])

In [ ]:
shap_values = explainer(whole_article) #Compute shap values

shap.plots.text(shap_values) #Visualize shap explanations

Token indices sequence length is longer than the specified maximum sequence length for this model (3915 > 1024). Running this sequence through the model will result in indexing errors


IndexError: ignored

In [ ]:
shap_values_abstract = explainer(abstract) #Compute shap values
shap.plots.text(shap_values_abstract) #Visualize shap explanations

Using `max_length`'s default (256) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.


  0%|          | 0/498 [00:00<?, ?it/s]

In [ ]:
shap.initjs()
shap.plots.text(shap_values_abstract) #Visualize shap explanations

In [ ]:
print(f"{shap.getjs()}{shap.plots.text(shap_values_abstract)}")

In [ ]:
# Define API function
def f(x):
    inputs = tokenizer(x.tolist(), return_tensors="pt", padding=True).to('cuda')
    with torch.no_grad():
        out = model.generate(**inputs)
    sentence = [tokenizer.decode(g, skip_special_tokens=True) for g in out]
    return np.array(sentence)

In [ ]:
# wrap model with TeacherForcingLogits class
model.to(device)
teacher_forcing_model = shap.models.TeacherForcing(f, similarity_model=model, similarity_tokenizer=tokenizer, device=model.device)
# create a Text masker
masker = shap.maskers.Text(tokenizer, mask_token = "...", collapse_mask_token=True)


Below is a demonstration of generating explanations for a model which is an api/function. Since this is a model agnostic case, we use a text similarity model to approximate log odds of generating output text which is used for computing shap explanations.

In [ ]:
explainer_model_agnostic = shap.Explainer(teacher_forcing_model, masker) #Create an explainer object using wrapped model and Text masker
shap_values_abstract_model_agnostic = explainer_model_agnostic(abstract)#Compute shap values
shap.initjs()

In [ ]:
function clickConnect() {
    console.log("Working");
    document.querySelector("colab-toolbar-button").click();
}
setInterval(clickConnect, 60000);

In [ ]:
display(shap.plots.text(shap_values_abstract_model_agnostic))